Different versions 

V1 - drop all the columns that represent the count elemnt in the describe function. Those column are reddundent
broght worse results than before

In [ ]:
train_ts = train_ts.drop(train_ts.iloc[:,1:10].columns,axis=1)
test_ts = test_ts.drop(test_ts.iloc[:,1:10].columns,axis=1)

V2 - Keep only highly correlated features to target inside the time series dataset

In [ ]:
id_and_target_train = pd.concat([Target_series, train_id], axis=1)

train_merged = id_and_target_train.merge(train_ts, on='id', how='left')
train_merged.drop('id',axis=1,inplace=True)
temp_corr = train_merged.corr()
#temp_corr["sii"].sort_values(ascending=False)

filtered_series = temp_corr["sii"][abs(temp_corr["sii"]) > 0.08]
row_names = filtered_series.index.tolist()
row_names.remove('sii')
train_ts[row_names]